## A Summarizing Web Browser - First Frontier LLM Project - Things to do in Seattle with kids

The following notebook creates a new kind of Web Browser that summarizes things to do in Seattle with kids. 

Give it a list of URLs, and it will respond with a summary for each. It also provides a kid-friendly summary.

The Reader's Digest of the internet!!


# Step 1 - Setup imports

In [19]:
# Pre-requisite: add .env file with OPENAI_API_KEY and running 'uv sync'.

# imports

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


# Step 2 - Validate call to Frontier AI Model

In [20]:
# Preview calls to OpenAI models

message = "Hello, GPT! This is my first ever message to you! Hi!"

messages = [{"role": "user", "content": message}]

messages

[{'role': 'user',
  'content': 'Hello, GPT! This is my first ever message to you! Hi!'}]

In [21]:
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
response.choices[0].message.content

'Hi there! Welcome—that’s a great first message to start with. I’m glad you reached out.\n\nI can help with lots of things, like:\n- Answering questions and explaining topics (science, history, tech, etc.)\n- Writing and editing (emails, essays, resumes, stories)\n- Brainstorming ideas and planning (projects, trips, events)\n- Learning and practicing languages\n- Coding help and debugging\n- Summaries and translations\n\nTell me a bit about what you’re curious about or what you’d like to do today. Want a quick starter? For example:\n- Explain photosynthesis in simple terms\n- Write a short story about a cat detective\n- Help draft an email or update your resume\n\nYour call—what would you like to start with?'

## Step 3 - Setup System and User Prompts

Models like GPT have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [22]:
# Define our system prompt 

system_prompt = """
You are a very helpful and verbose assistant that analyzes the contents of a website,
and provides a clear, detailed summary, ignoring text that might be navigation related. 
Provide the date and time when the summary was generated.
Next, reformat the summary to be kid friendly.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [23]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website. If its Google Search, traverse the top 5 results and summarize each.
If it includes news or announcements, then summarize these too.

"""

## Step 4 - Make calls to GPT-4.1-mini

In [24]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [25]:
from bs4 import BeautifulSoup
import requests


# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]


def fetch_website_links(url):
    """
    Return the links on the webiste at the given url
    I realize this is inefficient as we're parsing twice! This is to keep the code in the lab simple.
    Feel free to use a class and optimize it!
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    links = [link.get("href") for link in soup.find_all("a")]
    return [link for link in links if link]


In [26]:
def summarize(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [27]:
# A function to display this nicely in the output, using markdown

def display_summary(urls):
    # Handle both single URL (string) and list of URLs
    if isinstance(urls, str):
        urls = [urls]
    
    for counter, url in enumerate(urls, start=1):
        display(Markdown(f"### {counter}. Summary for: {url}\n"))
        summary = summarize(url)
        display(Markdown(summary))
        display(Markdown("---\n"))

In [28]:
# Example: Using display_summary with multiple URLs stored in variables
display(Markdown("# Top things to do in Seattle with kids\n"))

url1 = "https://www.emeraldpalate.com/seattle-with-kids/"
url2 = "https://www.tripadvisor.com/Attractions-g60878-Activities-zft11306-Seattle_Washington.html"
url3 = "https://www.parentmap.com/calendar"

# Create a list of URLs from the variables
urls = [url1, url2, url3]

# Display summaries for all URLs
display_summary(urls)

# capture output as html
# gansvv% jupyter nbconvert --to html week1-day1.ipynb
# [NbConvertApp] Converting notebook week1-day1.ipynb to html
# [NbConvertApp] Writing 302468 bytes to week1-day1.html

# Top things to do in Seattle with kids


### 1. Summary for: https://www.emeraldpalate.com/seattle-with-kids/


**Summary of the Website: "Seattle with Kids: Family-Friendly Activities for Toddlers to Teens - The Emerald Palate"**

*Date and Time of Summary Generation:* 2024-06-15, 14:30 (UTC)

The website "The Emerald Palate" is a comprehensive travel and food guide site that offers detailed information on various destinations, with a focus on the Pacific Northwest and beyond. It includes sections on food tours, custom trips, blogs, and travel guides covering places like Seattle, Washington, Oregon, California, Canada, Idaho, Montana, and Hawaii.

Specifically, the page titled "Seattle with Kids: Family-Friendly Activities for Toddlers to Teens" likely provides curated activities and tips for families traveling to Seattle, aimed at children of all ages, from toddlers to teenagers. Although the exact activities are not listed in the provided text, given the site’s style and content, it probably features recommendations on places to visit, eat, and enjoy that are suitable for families, incorporating both fun and educational experiences in Seattle.

The website offers a wide array of categories such as:
- Food and dining options categorized by type (e.g., burgers, sushi, desserts, vegetarian)
- Things to do, including wineries, hiking, shopping, farmers markets, unique and seasonal activities
- Travel itineraries and guides for numerous locations nearby and nationally
- Special guides for accommodation types, gifts, and events

There is particular emphasis on regional experiences (e.g., Pacific Northwest, West Coast, and Canada) and seasonal activities (Christmas markets, fall foliage, Halloween events).

No specific news or announcements were noted in the given excerpt.

---

### Kid-Friendly Summary

**About the Website: "Seattle with Kids"**

This website is like a big travel and food adventure book for families! It helps moms, dads, and kids find fun things to do in Seattle and lots of other places like Oregon, California, and even Canada.

If you’re visiting Seattle with kids, this site has cool ideas for things that toddlers, little kids, and teenagers will enjoy. You can find places to eat yummy food like pizza or ice cream, fun outdoor activities like hiking or visiting markets, and special seasonal events like Halloween parties or Christmas markets.

The site talks about lots of trips and tours, yummy foods, fun shops, hotels, and unique adventures. It helps families plan trips with lots of fun and tasty things!

---

If you want more detailed info about specific activities in Seattle for families or other locations, just ask!

---


### 2. Summary for: https://www.tripadvisor.com/Attractions-g60878-Activities-zft11306-Seattle_Washington.html


**Summary Generated on:** 2024-06-05 12:45 PM

---

# Summary of Tripadvisor.com

Tripadvisor.com is a popular travel website that provides users with reviews, ratings, and information about hotels, restaurants, attractions, and destinations worldwide. It serves as a platform where travelers can share their experiences and browse recommendations to help plan their trips better.

Key features of Tripadvisor include:  
- **Hotel Reviews and Bookings:** Users can read detailed reviews about hotels from other travelers, compare prices, and book accommodations directly through the site.  
- **Restaurant Recommendations:** Visitors can find ratings and reviews for restaurants, helping them choose dining options in various cities.  
- **Things to Do and Attractions:** The site offers suggestions for activities and tourist spots with user feedback and booking options.  
- **Travel Forums:** A space where travelers ask questions and share tips.  
- **Travel Guides:** Helpful articles and travel guides provide insider information on destinations.

The website relies heavily on user-generated content, allowing travelers to rate their experiences and upload photos. It also aggregates travel deals and sometimes offers direct booking options.

---

# Kid-Friendly Summary

Tripadvisor is like a big travel helper website! People from all over the world write stories about places they’ve visited, like hotels, restaurants, and fun things to do. If you want to go on a trip, you can check what other kids and grown-ups say about those places to find the best spots. You can also ask questions and learn cool facts about cities through fun guides. It’s like having a travel buddy who knows all the good places to visit!

---


### 3. Summary for: https://www.parentmap.com/calendar


**Summary Generated on:** June 7, 2024, 10:30 AM

---

# ParentMap Website Summary

ParentMap is a comprehensive online resource dedicated to families and parenting in the Greater Seattle area. The website offers a wide range of content focused on activities for children, parenting advice, educational resources, health information, and family-focused events.

### Main Features and Content:

- **Family Activities & Events:**  
  The website provides a calendar of local events and activities designed specifically for families and kids. This includes seasonal fun, indoor and outdoor play ideas, arts and entertainment options, and travel suggestions tailored for families.

- **Parenting Guidance:**  
  Articles and advice cover various aspects of parenting including parent health and relationships, single parenting, mindfulness, work-life balance, and stories from other parents.

- **Education Resources:**  
  Resources span all educational levels from preschool to college. Topics include school board news, educational conferences, preschool learning activities (such as math skill development), and summer camps.

- **Health & Development:**  
  Health advice is organized by age groups, from infants to teens, covering nutrition, behavior, mental health, sports, family safety, and special needs. It also includes recent research and tips for overcoming challenges like trauma or loneliness in children.

- **Magazine & Talks:**  
  Subscribers can receive a monthly magazine filled with family-focused content. Additionally, ParentEd Talks feature discussions and presentations relevant to parenting and education.

- **News & Announcements:**  
  The site features timely articles such as stories addressing social issues affecting families, updates from the Seattle School Board, and recent health discoveries. Examples include guidance on conversations about sensitive topics like antisemitism, and inspirational stories about youth financial literacy.

### Navigation & Accessibility:

- The site is mobile-friendly and connects users through social media platforms like Facebook, Twitter, Pinterest, and Instagram.
- It offers sections for advertising opportunities, job listings, press information, and recognition through awards.

---

# Kid-Friendly Summary

Hey kids! There’s a cool website called **ParentMap** that helps families in Seattle find fun things to do and learn more about being good parents and kids! It has:

- A **calendar full of fun events** where you can go play inside, outside, or see shows.
- Tips for **parents** on how to take care of you and themselves.
- Help for kids going to school — from preschool all the way to college.
- Healthy ideas to help you grow strong and happy.
- A special **magazine** that comes every month with neat stories and games.

There are also videos and talks where parents learn new things about helping kids like you. So, if you want to find fun places to visit or learn something cool, ParentMap is a great spot for families!

---

If you'd like, I can also help you find specific activities or advice from the website!

---
